In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torchvision.utils as vutils
import os


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, 512, 4, 1, 0, bias=False),   # 1x1 -> 4x4
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),  # 4x4 -> 8x8
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),  # 8x8 -> 16x16
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),   # 16x16 -> 32x32
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),     # 32x32 -> 64x64
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 2, 1, bias=False),  # Output shape (batch,1,4,4)

        )

    def forward(self, x):
        return self.main(x).view(x.size(0), -1).mean(1)  # Average over spatial dims

# WGAN loss
def generator_loss(fake_scores):
    return -fake_scores.mean()

def discriminator_loss(real_scores, fake_scores):
    return fake_scores.mean() - real_scores.mean()


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nz = 100  # latent vector size

netG = Generator(nz).to(device)
# Use this if running on cpu ↓
#netG.load_state_dict(torch.load('/content/drive/MyDrive/Generated/generator_model.pth', map_location=torch.device('cpu')))
netG.load_state_dict(torch.load('/content/drive/MyDrive/Generated/generator_model.pth'))
netG.eval()


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [7]:
# Generate new images
num_images = 100
noise = torch.randn(num_images, nz, 1, 1, device=device)
fake_images = netG(noise)


output_dir = '/content/drive/MyDrive/Generated/Synthdata'
os.makedirs(output_dir, exist_ok=True)

for i in range(num_images):
    path = os.path.join(output_dir, f"new_generated_{i+1}.png")
    vutils.save_image(fake_images[i], path, normalize=True)
    print(f"Saved new image {i+1} to {path}")


Saved new image 1 to /content/drive/MyDrive/Generated/Synthdata/new_generated_1.png
Saved new image 2 to /content/drive/MyDrive/Generated/Synthdata/new_generated_2.png
Saved new image 3 to /content/drive/MyDrive/Generated/Synthdata/new_generated_3.png
Saved new image 4 to /content/drive/MyDrive/Generated/Synthdata/new_generated_4.png
Saved new image 5 to /content/drive/MyDrive/Generated/Synthdata/new_generated_5.png
Saved new image 6 to /content/drive/MyDrive/Generated/Synthdata/new_generated_6.png
Saved new image 7 to /content/drive/MyDrive/Generated/Synthdata/new_generated_7.png
Saved new image 8 to /content/drive/MyDrive/Generated/Synthdata/new_generated_8.png
Saved new image 9 to /content/drive/MyDrive/Generated/Synthdata/new_generated_9.png
Saved new image 10 to /content/drive/MyDrive/Generated/Synthdata/new_generated_10.png
Saved new image 11 to /content/drive/MyDrive/Generated/Synthdata/new_generated_11.png
Saved new image 12 to /content/drive/MyDrive/Generated/Synthdata/new_gen